# Model Building Setup for Safety Models using RdKit descriptors and ChEMBL31 data only

# Setup

In [ ]:
# input
model_dir = "ADRA2A_PIC50"
data_file_name = "./ADRA2A_PIC50_train.tsv"
# output
scalar_file_name = "./scalar_ADRA2A_PIC50.pkl"
ecc_model_file = "./ADRA2A_PIC50_svm.pkl"
brf_model_file = "./ADRA2A_PIC50_xgb.pkl"
rusb_model_file = "./ADRA2A_PIC50_rusb.pkl"

In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np
import pandas as pd
import joblib
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, roc_auc_score, balanced_accuracy_score, matthews_corrcoef, cohen_kappa_score 

seed = 1234

In [2]:
from imblearn.ensemble import EasyEnsembleClassifier,BalancedRandomForestClassifier,RUSBoostClassifier

# Get Mols, Data and generate descriptors

In [3]:
df = pd.read_csv(data_file_name, sep="\t")

In [4]:
y_str = df.class_label.tolist() 
y_int = pd.get_dummies(y_str)
y_new = y_int["POSITIVE"] 
df['class_label_binary'] = y_new 
y = df.class_label_binary.to_list()

In [5]:
mols = [Chem.MolFromSmiles(smi) for smi in df.parentised_smiles]
# generate binary Morgan fingerprint with radius 2
fp = [AllChem.GetMorganFingerprintAsBitVect(m, 2) for m in mols]
# generate binary Morgan fingerprint with radius 2 with physchem as numpy array
x = rdkit_fpconvert_numpy(fp)
x = np.concatenate((x, rdkit_get_physchem_descr(mols)), axis=1)

## Scale using scalar from last run and calculate splits

In [6]:
# randomly select 20% of compounds as test set but with stratified selection
x_tr, x_ts, y_tr, y_ts = train_test_split(x, y, test_size=0.20, random_state=seed, stratify=y)

In [ ]:
# save scalar
scale = joblib.load(scalar_file_name)

In [9]:
x_tr = scale.transform(x_tr)
x_ts = scale.transform(x_ts)

# IMBLearn Models

In [10]:
# create grid search dictionary
eec_param_grid = {"n_estimators": [i for i in [10, 30, 50, 100]]}
# model build
eec = GridSearchCV(EasyEnsembleClassifier(n_jobs=-1),
                  verbose=2,
                  scoring='balanced_accuracy',
                  param_grid=eec_param_grid, cv=5,
                  n_jobs=-1)
eec.fit(x_tr, y_tr)
# save model
joblib.dump(eec, ecc_model_file, compress=3)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................................n_estimators=10; total time=   1.7s
[CV] END ....................................n_estimators=10; total time=   1.7s
[CV] END ....................................n_estimators=10; total time=   1.7s
[CV] END ....................................n_estimators=10; total time=   1.7s
[CV] END ....................................n_estimators=10; total time=   1.7s
[CV] END ....................................n_estimators=30; total time=   5.2s
[CV] END ....................................n_estimators=30; total time=   5.2s
[CV] END ....................................n_estimators=30; total time=   5.1s
[CV] END ....................................n_estimators=30; total time=   5.2s
[CV] END ....................................n_estimators=30; total time=   5.2s
[CV] END ....................................n_estimators=50; total time=   8.6s
[CV] END ....................................n_es

['./ADRA2A_PIC50_eec.pkl']

In [11]:
# create grid search dictionary
brf_param_grid = {"n_estimators": [i for i in [10, 30, 50, 100]]}
# model build
brf = GridSearchCV(BalancedRandomForestClassifier(n_jobs=-1, replacement=True, sampling_strategy='all'),
                  verbose=2,
                  scoring='balanced_accuracy',
                  param_grid=eec_param_grid, cv=5,
                  n_jobs=-1)
brf.fit(x_tr, y_tr)
# save model
joblib.dump(brf, brf_model_file, compress=3)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................................n_estimators=10; total time=   0.0s
[CV] END ....................................n_estimators=10; total time=   0.0s
[CV] END ....................................n_estimators=10; total time=   0.0s
[CV] END ....................................n_estimators=10; total time=   0.0s
[CV] END ....................................n_estimators=10; total time=   0.0s
[CV] END ....................................n_estimators=30; total time=   0.1s
[CV] END ....................................n_estimators=30; total time=   0.1s
[CV] END ....................................n_estimators=30; total time=   0.1s
[CV] END ....................................n_estimators=30; total time=   0.1s
[CV] END ....................................n_estimators=30; total time=   0.1s
[CV] END ....................................n_estimators=50; total time=   0.1s
[CV] END ....................................n_es

['./ADRA2A_PIC50_brf.pkl']

In [12]:
# create grid search dictionary
rusb_param_grid = {"n_estimators": [i for i in [10, 30, 50, 100]]}
# model build
rusb = GridSearchCV(RUSBoostClassifier(),
                  verbose=2,
                  scoring='balanced_accuracy',
                  param_grid=eec_param_grid, cv=5,
                  n_jobs=-1)
rusb.fit(x_tr, y_tr)
# save model
joblib.dump(eec, rusb_model_file, compress=3)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
[CV] END ....................................n_estimators=10; total time=   0.1s
[CV] END ....................................n_estimators=10; total time=   0.1s
[CV] END ....................................n_estimators=10; total time=   0.1s
[CV] END ....................................n_estimators=10; total time=   0.1s
[CV] END ....................................n_estimators=10; total time=   0.1s
[CV] END ....................................n_estimators=30; total time=   0.2s
[CV] END ....................................n_estimators=30; total time=   0.2s
[CV] END ....................................n_estimators=30; total time=   0.2s
[CV] END ....................................n_estimators=30; total time=   0.2s
[CV] END ....................................n_estimators=30; total time=   0.2s
[CV] END ....................................n_estimators=50; total time=   0.3s
[CV] END ....................................n_es

['./ADRA2A_PIC50_rusb.pkl']

## Test Set Validation

In [13]:
# predict for the test set compounds
pred_eec_tr = eec.predict(x_tr)
# calc statistics
print("EEC Train:")
print("Precision = ", precision_score(y_tr, pred_eec_tr))
print("ROC-AUC = ", roc_auc_score(y_tr, pred_eec_tr))
print("Balanced Accuracy Score = ", balanced_accuracy_score(y_tr, pred_eec_tr))
print("MCC = ", matthews_corrcoef(y_tr, pred_eec_tr))
print("Kappa = ", cohen_kappa_score(y_tr, pred_eec_tr))

EEC Train:
Precision =  1.0
ROC-AUC =  0.9060457516339869
Balanced Accuracy Score =  0.9060457516339869
MCC =  0.6225665639589993
Kappa =  0.5586511441188278


In [14]:
# predict for the test set compounds
pred_eec_ts = eec.predict(x_ts)
# calc statistics
print("EEC Test:")
print("Precision = ", precision_score(y_ts, pred_eec_ts))
print("ROC-AUC = ", roc_auc_score(y_ts, pred_eec_ts))
print("Balanced Accuracy Score = ", balanced_accuracy_score(y_ts, pred_eec_ts))
print("MCC = ", matthews_corrcoef(y_ts, pred_eec_ts))
print("Kappa = ", cohen_kappa_score(y_ts, pred_eec_ts))

EEC Test:
Precision =  0.9916666666666667
ROC-AUC =  0.8671328671328672
Balanced Accuracy Score =  0.8671328671328671
MCC =  0.547562941427901
Kappa =  0.47572815533980584


In [15]:
# calc statistics
# predict for the test set compounds
pred_brf_tr = brf.predict(x_tr)
# calc statistics
print("BRF Train:")
print("Precision = ", precision_score(y_tr, pred_brf_tr))
print("ROC-AUC = ", roc_auc_score(y_tr, pred_brf_tr))
print("Balanced Accuracy Score = ", balanced_accuracy_score(y_tr, pred_brf_tr))
print("MCC = ", matthews_corrcoef(y_tr, pred_brf_tr))
print("Kappa = ", cohen_kappa_score(y_tr, pred_brf_tr))

BRF Train:
Precision =  1.0
ROC-AUC =  0.928921568627451
Balanced Accuracy Score =  0.928921568627451
MCC =  0.6849328183743822
Kappa =  0.6386528335331838


In [16]:
# calc statistics
# predict for the test set compounds
pred_brf_ts = brf.predict(x_ts)
# calc statistics
print("BRF Train:")
print("Precision = ", precision_score(y_ts, pred_brf_ts))
print("ROC-AUC = ", roc_auc_score(y_ts, pred_brf_ts))
print("Balanced Accuracy Score = ", balanced_accuracy_score(y_ts, pred_brf_ts))
print("MCC = ", matthews_corrcoef(y_ts, pred_brf_ts))
print("Kappa = ", cohen_kappa_score(y_ts, pred_brf_ts))

BRF Train:
Precision =  0.9761904761904762
ROC-AUC =  0.8416583416583416
Balanced Accuracy Score =  0.8416583416583416
MCC =  0.5241877966925762
Kappa =  0.4720496894409938


In [17]:
# calc statistics
# predict for the test set compounds
pred_rusb_tr = rusb.predict(x_tr)
# calc statistics
print("RUSB Train:")
print("Precision = ", precision_score(y_tr, pred_rusb_tr))
print("ROC-AUC = ", roc_auc_score(y_tr, pred_rusb_tr))
print("Balanced Accuracy Score = ", balanced_accuracy_score(y_tr, pred_rusb_tr))
print("MCC = ", matthews_corrcoef(y_tr, pred_rusb_tr))
print("Kappa = ", cohen_kappa_score(y_tr, pred_rusb_tr))

RUSB Train:
Precision =  0.9320388349514563
ROC-AUC =  0.7254901960784313
Balanced Accuracy Score =  0.7254901960784313
MCC =  0.35444489141573904
Kappa =  0.32616953387469827


In [18]:
# calc statistics
# predict for the test set compounds
pred_rusb_ts = rusb.predict(x_ts)
# calc statistics
print("RUSB Train:")
print("Precision = ", precision_score(y_ts, pred_rusb_ts))
print("ROC-AUC = ", roc_auc_score(y_ts, pred_rusb_ts))
print("Balanced Accuracy Score = ", balanced_accuracy_score(y_ts, pred_rusb_ts))
print("MCC = ", matthews_corrcoef(y_ts, pred_rusb_ts))
print("Kappa = ", cohen_kappa_score(y_ts, pred_rusb_ts))

RUSB Train:
Precision =  0.9083969465648855
ROC-AUC =  0.6555944055944056
Balanced Accuracy Score =  0.6555944055944056
MCC =  0.24577478229336402
Kappa =  0.22753834915997084
